# Ranked Choice Voting

In [7]:
import numpy as np
import random

import pandas as pd
pd.set_option('display.width',1000)

## Generate Sample Data

This function generates random ranking of candidates (including randomizing number of candidates ranked).

In [3]:
# SET CONSTANTS

# CANDIDATE POOL
CANDIDATES = ['Zion', 'Yosemite', 'Mt Rainier', 'Yellowstone', 'Glacier', 'Rocky', 'Grand Canyon']
# VOTING POPULATION SIZE
N = 1000

In [38]:
def generate_ballots(n, candidates, verbose=False):
    max_ranking = len(candidates)

    ballots_dict = {}

    for i in range(0, n):
        # randomly decide how many of candidates to rank
        ranked = np.random.randint(0, max_ranking+1)
    
        # choose that many candidates randomly
        order = random.sample(candidates, ranked)

        if verbose:
            print(f'Voter {i}: {order}')

        ballots_dict[i] = dict(zip(range(1, len(order)+1), order))

    return pd.DataFrame.from_dict(ballots_dict, orient='index').melt(ignore_index=False, var_name='ranking', value_name='candidate').reset_index(names='voter_id')

### Verbose Example

In [46]:
sample = generate_ballots(10, CANDIDATES, True)

Voter 0: ['Zion', 'Rocky']
Voter 1: ['Mt Rainier', 'Glacier', 'Yellowstone', 'Yosemite', 'Rocky']
Voter 2: []
Voter 3: ['Mt Rainier', 'Rocky', 'Zion', 'Glacier']
Voter 4: ['Yosemite', 'Grand Canyon', 'Mt Rainier']
Voter 5: ['Rocky', 'Glacier', 'Yosemite', 'Mt Rainier', 'Zion']
Voter 6: ['Yellowstone']
Voter 7: ['Yellowstone', 'Rocky', 'Glacier', 'Mt Rainier', 'Grand Canyon']
Voter 8: ['Grand Canyon', 'Mt Rainier', 'Yosemite', 'Glacier', 'Zion', 'Yellowstone']
Voter 9: ['Grand Canyon', 'Mt Rainier']


In [41]:
np_ballots = generate_ballots(1000, CANDIDATES)

## Tabulate Ballots

In [23]:
def tabulate_ballots(ballots, candidates, threshold, verbose=False):

    # drop all empty ranks (candidate is null)
    ballots = ballots.loc[ballots['candidate'].notnull()]
    
    # initialize vote share
    candidate_vote_share = pd.Series(0, index=candidates)

    # initialize remaining candidates list
    cand_remain = set(candidates)

    rnd = 1

    while candidate_vote_share.max() < threshold:

        # calculate vote share of current tier
        candidate_vote_share = ballots.loc[ballots['ranking'] == 1, 'candidate'].value_counts(normalize=True)

        if verbose:
            print(f'\nRound {rnd} Candidate Vote Share:')
            print(candidate_vote_share)

            print(pd.pivot_table(data=ballots, index='voter_id', values='candidate', columns='ranking', aggfunc='min'))

        if candidate_vote_share.max() < threshold:
            # if no candidates meet threshold of voter share
            # eliminate candidate with lowest share, as well as candidates who got no vote share

            # eliminate candidates who got 0 votes
            cand_dq = cand_remain - set(candidate_vote_share.index)
            dq = ballots.loc[ballots['candidate'].isin(cand_dq)].shape[0]
            ballots = ballots.loc[~ballots['candidate'].isin(cand_dq)]

            if verbose and len(cand_dq) > 0:
                print(f'Candidates with 0 1st place votes:\t {cand_dq}\n Redistributed {dq} lower-place votes')

            # get candidate with lowest share
            cand_elim = candidate_vote_share.idxmin()
            # store number of voters who ranked this candidate (in any slot)
            elim = ballots.loc[ballots['candidate'] == cand_elim].shape[0]
            ballots = ballots.loc[ballots['candidate'] != cand_elim]

            new_col_name = 'ranking_' + str(rnd)
            ballots[new_col_name] = ballots['ranking']

            # re-rank candidates for each voter
            ballots['ranking'] = ballots.groupby('voter_id')['ranking'].rank()

            if verbose:
                print(f'Last-place candidate:\t {cand_elim} with {candidate_vote_share.min():.1%}\n Redistributed {elim} votes')

            rnd += 1
            cand_remain = set(ballots['candidate'].unique())

    return candidate_vote_share

    

### Concise Example (default)

In [27]:
tabulate_ballots(np_ballots, CANDIDATES, 0.6)

candidate
Rocky    0.666667
Zion     0.333333
Name: proportion, dtype: float64

### Verbose Example

In [29]:
tabulate_ballots(np_ballots, CANDIDATES, 0.6, verbose=True)


Round 1 Candidate Vote Share:
candidate
Zion            0.333333
Rocky           0.333333
Glacier         0.166667
Grand Canyon    0.166667
Name: proportion, dtype: float64
ranking              1             2             3         4             5           6     7
voter_id                                                                                    
0                 Zion   Yellowstone    Mt Rainier     Rocky  Grand Canyon         NaN   NaN
1              Glacier         Rocky          Zion  Yosemite           NaN         NaN   NaN
5                 Zion    Mt Rainier  Grand Canyon       NaN           NaN         NaN   NaN
7                Rocky  Grand Canyon   Yellowstone  Yosemite          Zion     Glacier   NaN
8                Rocky      Yosemite   Yellowstone      Zion           NaN         NaN   NaN
9         Grand Canyon      Yosemite       Glacier     Rocky   Yellowstone  Mt Rainier  Zion
Candidates with 0 1st place votes:	 {'Yellowstone', 'Mt Rainier', 'Yosemite'}
 Red

candidate
Rocky    0.666667
Zion     0.333333
Name: proportion, dtype: float64